In [2]:
import mujoco
from meshcat_mujoco import MeshCatVisualizer
import numpy as np

In [3]:
xml_path = './a1/xml/a1.xml'
model    = mujoco.MjModel.from_xml_path(xml_path)
data     = mujoco.MjData(model)

In [ ]:
viewer = MeshCatVisualizer('./a1/xml/a1.xml', model, data)
viewer.jupyter_cell()